In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import
 re
import tensorflow as tf
import gensim
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
fake = pd.read_csv("/content/Fake.csv")
fake.head()
real = pd.read_csv("/content/True.csv")
real.head(10) 

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [32]:
for index,row in enumerate(real.text.values):
      record = row.split(" -", maxsplit=1)
      if(len(record) != 1 and len(record[0]) < 260):
        real.text[index] = record[1]
      if(str(row).strip() == '' or len(row) > 800):
        real = real.drop(index)

for index,row in enumerate(fake.text.values):
      record = row.split(" -", maxsplit=1)
      if(len(record) != 1 and len(record[0]) < 260):
        fake.text[index] = record[1]
      if(str(row).strip() == ''or len(row) > 800):
        fake = fake.drop(index)


In [33]:
real["class"] = 1
fake["class"] = 0

real.head()

real["text"] = real["title"] + " " + real["text"]
fake["text"] = fake["title"] + " " + fake["text"]

real = real.drop(["subject", "date","title"], axis=1)
fake = fake.drop(["subject", "date", "title"], axis=1)

data = real.append(fake, ignore_index=True)

In [34]:
data.head()
m


In [35]:
w2v_model = gensim.models.Word2Vec(sentences=x, size=100, window=5, min_count=1)

w2v_model.wv.most_similar("fbi")

[('director', 0.9804056882858276),
 ('comey', 0.9684277772903442),
 ('kallstrom', 0.9549883604049683),
 ('asst', 0.9548935294151306),
 ('james', 0.9428326487541199),
 ('scorch', 0.9305459260940552),
 ('former', 0.9183748960494995),
 ('directors', 0.9167969226837158),
 ('murphy', 0.9140491485595703),
 ('nsa', 0.9059329628944397)]

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)

x = pad_sequences(x, maxlen=800)

In [52]:
vocab_size = len(tokenizer.word_index) + 1

print(np.asarray().shape)





(20492, 100)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
layers = tf.keras.layers
model = tf.keras.models.Sequential()
model.add(layers.Embedding(len(tokenizer.word_index)+1, output_dim=100, weights=[np.append(w2v_model.wv.syn0,(np.zeros((1, 100))), axis = 0)], input_length=800, trainable=False))
model.add(layers.LSTM(units=128))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

xtrain, xtest, ytrain, ytest = train_test_split(x, y) 

model.fit(xtrain, ytrain, epochs = 6, validation_split=0.1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/6
158/158 [==============================] - 121s 755ms/step - loss: 0.4942 - acc: 0.7554 - val_loss: 0.2582 - val_acc: 0.9057
Epoch 2/6
158/158 [==============================] - 120s 760ms/step - loss: 0.2328 - acc: 0.9126 - val_loss: 0.2035 - val_acc: 0.9306
Epoch 3/6
158/158 [==============================] - 119s 756ms/step - loss: 0.1549 - acc: 0.9460 - val_loss: 0.1220 - val_acc: 0.9573
Epoch 4/6
158/158 [==============================] - 119s 754ms/step - loss: 0.1329 - acc: 0.9516 - val_loss: 0.1037 - val_acc: 0.9609
Epoch 5/6
158/158 [==============================] - 118s 747ms/step - loss: 0.1140 - acc: 0.9580 - val_loss: 0.0851 - val_acc: 0.9680
Epoch 6/6
158/158 [==============================] - 118s 745ms/step - loss: 0.0852 - acc: 0.9718 - val_loss: 0.0727 - val_acc: 0.9751


In [55]:
from sklearn.metrics import accuracy_score
ypred = (model.predict(xtest) >= 0.5).astype("int")

accuracy_score(ytest, ypred)

0.9754142169962586

In [56]:
print(xtest[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   